## Set-up

In [22]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from pprint import pprint
import re
from tqdm import tqdm
tqdm.pandas()
import numpy as np

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=12, progress_bar=True)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [23]:
#!pwd
import os

# Google Bucket
# file name checkpoint_0512_sent_split.parquet
path_bucket = 'gs://msca-sp23-bucket/nlp_data'
bucket_read = path_bucket + '/' + 'checkpoint_0523_cleaned_filtered_data.parquet'
runtime_path = '/home/anthony/projects/nlp_runtime'

os.chdir(runtime_path)
print(os.getcwd())

/home/anthony/projects/nlp_runtime


In [24]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

import warnings
warnings.filterwarnings("ignore")

In [25]:
import gensim
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

In [26]:
import nltk as nltk
from nltk.corpus import stopwords

import multiprocessing

In [27]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_md")
nlp = spacy.load("en_core_web_md")

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 25.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [28]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/anthony/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/anthony/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/anthony/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/anthony/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/anthony/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
# read data
df_raw = pd.read_parquet(bucket_read, engine='pyarrow')

In [ ]:
# take a sample
df = df_raw.sample(50000, random_state=42)

print(f'raw data shape{df_raw.shape}')
print(f'sample shape{df.shape}')
df.head(1)

raw data shape(185479, 4)
sample shape(50000, 4)


,url,date,title,text_cleaned
95601,https://www.wkyt.com/prnewswire/2022/06/14/windwards-first-of-its-kind-ai-model-creates-new-standard-eta-prediction-accuracy-critical-mitigating-supply-chain-disruptions/,2022-06-14,Windward's First-of-its-Kind AI Model Creates a New Standard of ETA Prediction Accuracy Critical to Mitigating Supply Chain Disruptions,Windward's First of its Kind AI Model Creates a New Standard of ETA Prediction Accuracy Critical to Mitigating Supply Chain Disruptions Skip to NewscastsEveryday Health DivideWKYT CamsClosings Championship 2022All BlueHigh School Game TimeMingua s Athlete of the WeekSports s Corner Off the BenchContact UsSubmit a News TipMeet the TeamAdvertising InformationJobs at WKYTNews AlertsSubmit Photos and VideosGray DC BureauInvestigate TVCommunityAloha MondaysBG H G TV of KindnessCommuni...


## LDA, on overall topics

### Text Prep

I want to utilize parallelization as much as possile to save time

Helper Functions

In [ ]:
%%time

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

# Define functions for stopwords, bigrams, trigrams and lemmatization
stop_words = stopwords.words('english')
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

CPU times: user 181 µs, sys: 2.67 ms, total: 2.85 ms
Wall time: 2.01 ms


In [ ]:
# select the text
df_text = df[['text_cleaned']]
#df_title = df['title']

# remove punctuation and numbers using parallel_apply
df_text['text_cleaned'] = df_text['text_cleaned'].parallel_apply(lambda x: re.sub('[^A-Za-z]+', ' ', x))
#df_title['title_cleaned'] = df_title.parallel_apply(lambda x: re.sub('[^A-Za-z]+', ' ', x))

In [ ]:
# drop na and duplicates
df_text = df_text.dropna().drop_duplicates()
# convert to str type
df_text['text_cleaned'] = df_text['text_cleaned'].astype(str)
df_text.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49810 entries, 95601 to 127958
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   text_cleaned  49810 non-null  object
dtypes: object(1)
memory usage: 778.3+ KB


In [ ]:
# save a copy
df_text_before = df_text.copy()

# define a function to handle errors
def handle_errors(func):
    def wrapper(x):
        try:
            return func(x)
        except Exception as e:
            print(f"Error processing row: {x}")
            return np.nan
    return wrapper

# define the remove_stopwords function with the handle_errors decorator
@handle_errors
def remove_stopwords(row): 
    return [i for i in simple_preprocess(row) if i not in stopwords.words('english')]

# apply remove_stopwords function with try/except
df_text['text_cleaned'] = df_text['text_cleaned'].parallel_apply(remove_stopwords)

In [ ]:
df_text.head()

,text_cleaned
95601,"[windward, first, kind, ai, model, creates, new, standard, eta, prediction, accuracy, critical, mitigating, supply, chain, disruptions, skip, health, dividewkyt, camsclosings, championship, bluehigh, school, game, timemingua, athlete, weeksports, corner, benchcontact, ussubmit, news, tipmeet, teamadvertising, informationjobs, wkytnews, alertssubmit, photos, videosgray, dc, mondaysbg, tv, starssponsored, storieswkyt, summer, grillin, sponsored, pepsitv, listingscircle, country, music, lifesty..."
176291,"[engineer, claimed, google, ai, thoughts, feelings, placed, leave, newsbreaksign, arttv, seriesbooks, dancebehind, viral, artstv, musichip, healthhealth, servicesmental, healthdiseases, sportspremier, safetypublic, advicefamily, rentlabor, issuestrouble, scienceearth, nationsmiddle, locations, channels, topics, people, inwate, follow, followers, post, viewsaboutwate, side, provides, latest, news, weather, sports, coverage, knoxville, east, tennessee, vitelloin, article, google, ai, software,..."
51870,"[flex, logix, announces, eflx, efpga, nnmax, ai, inference, ip, model, support, veloce, strato, emulation, platform, mentor, home, search, silicon, ip, search, verification, ip, latest, news, industry, articles, industry, expert, blogs, videos, slides, menu, design, reuse, subscribe, soc, news, alert, design, reuse, silicon, ip, analog, mixed, signal, storage, controller, phy, graphic, peripheral, interface, controller, phy, processors, memory, logic, library, security, multimedia, wireline,..."
183511,"[vinsolutions, virtual, assistant, vinessa, uses, ai, help, salespeople, pursue, convert, leads, tight, market, resources, blog, journalists, log, sign, data, privacy, typing, field, list, search, results, appear, automatically, updated, type, searching, content, results, found, please, change, search, terms, try, news, releases, public, company, english, auto, automotive, transportation, aerospace, defense, air, freight, airlines, aviation, automotive, maritime, shipbuilding, railroads, int..."
131260,"[nab, adobe, unveils, plans, ai, tools, video, news, broadcast, site, uses, cookies, using, site, agreeing, privacy, cookie, policy, skip, main, contentskip, navigation, hot, survey, ratingssvods, broadcast, mast, search, site, search, site, back, parent, navigation, item, ratings, back, parent, navigation, item, premium, back, parent, navigation, item, features, back, parent, navigation, item, international, homebroadcast, awards, best, places, work, tv, indie, survey, broadcast, digital, a..."


### titleza LDA

In [ ]:
%%time
# tokenize the text
data_list = df_text['text_cleaned'].tolist()
data_tokens = list(sent_to_words(data_list))

CPU times: user 2min 1s, sys: 1.48 s, total: 2min 3s
Wall time: 2min 3s


In [20]:
%%time
# create bigrams & trigrams
bigram = gensim.models.Phrases(data_tokens, min_count=1, threshold=1)
trigram = gensim.models.Phrases(bigram[data_tokens], threshold=1)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

CPU times: user 3min 34s, sys: 1.94 s, total: 3min 36s
Wall time: 3min 36s


In [21]:
%%time
# Remove Stop Words
#data_tokens_nostops = remove_stopwords(data_tokens)

# Create n-grams
data_words_bigrams = make_bigrams(data_tokens)
data_words_trigrams = make_trigrams(data_tokens)

# Combine tokens and n-grams
# data_tokens_cobnined = data_tokens_nostops + data_words_bigrams + data_words_trigrams
data_tokens_cobnined = data_words_trigrams

# Lemmatize text keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_tokens_cobnined, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(*data_lemmatized[:1])

KeyboardInterrupt: 

In [ ]:
checkpoint_path = '/home/anthony/projects/nlp_runtime/'

# save the limmatized data to txt
path = checkpoint_path + '/' + 'data_lemmatized.txt'
with open(path, 'w') as f:
    for item in data_lemmatized:
        f.write("%s\n" % item)

In [ ]:
# # read the limmatized data from txt
# path = checkpoint_path + '/' + 'data_lemmatized.txt'
# with open(path, 'r') as f:
#     data_read = f.read().splitlines()

# read each element (a str) of data_read to a list; append to data_lemmatized
data_lemmatized = []

def string_to_list(str):
    # revemo al " and []
    str = str.replace('"', '')
    str = str.replace('\'', '')
    str = str.replace('[', '')
    str = str.replace(']', '')
    # split by ', '
    output_list = str.split(', ')
    return output_list

for i in range(len(data_read)):
    data_lemmatized.append(string_to_list(data_read[i]))

In [ ]:
%%time

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(data_lemmatized)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in data_lemmatized]

In [ ]:
num_processors = multiprocessing.cpu_count()
workers = num_processors-1
print(f'Using {workers} workers')

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = LdaMulticore(corpus=doc_term_matrix,
                       id2word=dictionary,
                       num_topics=k,
                       random_state=100,                  
                       passes=10,
                       alpha=a,
                       eta=b,
                       workers=workers)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 10
step_size = 1
topics_range = range(min_topics, max_topics+1, step_size)

# Alpha parameter
alpha = ['asymmetric'] # Run for number of topics only

# Beta parameter
beta = ['auto'] # Run for number of topics only

# Validation sets
num_of_docs = len(doc_term_matrix)
corpus_sets = [# gensim.utils.ClippedCorpus(doc_term_matrix, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(doc_term_matrix, num_of_docs*0.5), 
               # gensim.utils.ClippedCorpus(doc_term_matrix, num_of_docs*0.75), 
               doc_term_matrix]

corpus_title = ['100% Corpus']
model_results = {
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

In [ ]:
'''%%time 

itr = 0
itr_total = len(beta)*len(alpha)*len(topics_range)*len(corpus_title)
print(f'LDA will execute {itr_total} iterations')

# iterate through hyperparameters
for i in tqdm(range(len(corpus_sets))):
    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        #tic()
        for a in alpha:
            # iterare through beta values
            for b in beta:
                # get the coherence score for the given parameters
                itr += 1
                cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dictionary,
                                              k=k, a=a, b=b)
                # Save the model results
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
                pct_completed = round((itr / itr_total * 100),1)
        print(f'Completed model based on {k} LDA topics. Finished {pct_completed}% of LDA runs')

lda_tuning = pd.DataFrame(model_results)     '''

In [ ]:
from joblib import Parallel, delayed

def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = LdaMulticore(corpus=corpus,
                       id2word=dictionary,
                       num_topics=k,
                       random_state=100,                  
                       passes=10,
                       alpha=a,
                       eta=b,
                       workers=workers)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
    return coherence_model_lda.get_coherence()

itr = 0
itr_total = len(beta)*len(alpha)*len(topics_range)*len(corpus_title)
print(f'LDA will execute {itr_total} iterations')

# iterate through hyperparameters
for i in tqdm(range(len(corpus_sets))):
    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        #tic()
        for a in alpha:
            # iterate through beta values
            jobs = []
            for b in beta:
                itr += 1
                # run each beta value in parallel
                job = delayed(compute_coherence_values)(corpus=corpus_sets[i], dictionary=dictionary,
                                                  k=k, a=a, b=b)
                jobs.append(job)
            # compute coherence scores in parallel and save results
            coherences = Parallel(n_jobs=min(8, len(jobs)))(jobs)
            for b, cv in zip(beta, coherences):
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
            pct_completed = round((itr / itr_total * 100),1)
        print(f'Completed model based on {k} LDA topics. Finished {pct_completed}% of LDA runs')

lda_tuning = pd.DataFrame(model_results)


In [ ]:
# Best LDA parameters
lda_tuning.sort_values(by=['Coherence'], ascending=False).head(10)

In [ ]:
lda_tuning.plot(x ='Topics', y='Coherence', kind = 'line', xticks=range(1,20))

In [ ]:
lda_tuning_best = lda_tuning.sort_values(by=['Coherence'], ascending=False).head(1)


tuned_topics = int(lda_tuning_best['Topics'].to_string(index=False))


# Since the values for Alpha and Beta can be float, symmetric and asymmetric, we will either strip or convert to float
try:
    tuned_alpha = float(lda_tuning_best['Alpha'].to_string(index=False))
except:
    tuned_alpha = lda_tuning_best['Alpha'].to_string(index=False).strip()
    

try:
    tuned_beta = float(lda_tuning_best['Beta'].to_string(index=False))
except:
    tuned_beta = lda_tuning_best['Beta'].to_string(index=False).strip()    
    
print(f'Best Parameters: Topics: {tuned_topics}, Alpha: {tuned_alpha}, Beta: {tuned_beta}')

In [ ]:
%%time

tuned_lda_model = LdaMulticore(corpus=doc_term_matrix,
                       id2word=dictionary,
                       #num_topics=tuned_topics,
                       num_topics=4,
                       random_state=100,
                       passes=10,
                       alpha=tuned_alpha,
                       eta=tuned_beta,
                       workers = workers)

coherence_model_lda = CoherenceModel(model=tuned_lda_model, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
print('\nCoherence Score: ', coherence_model_lda)

In [ ]:
%%time

lda_display = gensimvis.prepare(tuned_lda_model, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)
